In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [1]:
!which python

/Users/fyenne/miniforge3/envs/siming/bin/python


---

In [2]:
path = '/Users/fyenne/Downloads/booooks/semester5/pythonAAE/open_data_com/data'
train = pd.read_csv(path+'/train.csv')
tests = pd.read_csv(path+'/test.csv')
sub = pd.read_csv(path+'/sub.csv')

train['year'], train['month'], train['date'] = 0,0,0
tests['year'], tests['month'], tests['date'] = 0,0,0
train.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in train['日期']])
tests.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in tests['日期']])
train['sum'] = train['A厂'] + train['B厂']
tests['sum'] = tests['A厂'] + tests['B厂']
train['frac_a'] = train['A厂']/train['sum']
tests['frac_a'] = tests['A厂']/tests['sum']
train['frac_b'] = train['B厂']/train['sum']
tests['frac_b'] = tests['B厂']/tests['sum']
train = train.drop('日期', axis = 1)
tests = tests.drop('日期', axis = 1)

In [3]:
dif_col = set(train.columns.to_list()) - set(pd.DataFrame(train.describe()).columns.to_list())

In [4]:
train['year'], train['month'], train['date'] = [train[i].astype(int) for i in list(dif_col)]
tests['year'], tests['month'], tests['date'] = [tests[i].astype(int) for i in list(dif_col)]

---
# baseline

In [357]:
bs_train = train.iloc[:,0:5]
bs_tests = tests.iloc[:,2:5]

In [358]:
bs_tar = bs_train.iloc[:, 0:2]
bs_train = bs_train.iloc[:, 2:]

In [35]:
from sklearn.linear_model import Ridge, LinearRegression
from sklearn import model_selection
cv_split = model_selection.ShuffleSplit(n_splits = 5, test_size = .2, train_size = .8, random_state = 1026529)


In [360]:
alg1 = LinearRegression()
cv_results_1 = model_selection.cross_validate(alg1, bs_train, bs_tar, cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_1['test_score'].mean()

-0.14551133194229993

In [361]:
alg2 = Ridge()
cv_results_2 = model_selection.cross_validate(alg2, bs_train, bs_tar, cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_2['test_score'].mean()

-0.1455161156470318

In [364]:
bs_mode=alg2.fit(bs_train, y = bs_tar)

In [373]:
sub.iloc[:,1:3] = pd.DataFrame(bs_mode.predict(bs_tests))/100

In [374]:
sub.to_csv('./data/baseline_sub.csv', index=False)

---

## test 2 , the pseudo data

In [27]:
import pandas as pd
import numpy as np
path = '/Users/fyenne/Downloads/booooks/semester5/pythonAAE/open_data_com/data'
train = pd.read_csv(path+'/train.csv')
tests = pd.read_csv(path+'/test.csv')
bssub = pd.read_csv(path+'/baseline_sub.csv')

In [3]:
tests.iloc[:, 1:] = bssub.iloc[:, 1:]
train['year'], train['month'], train['date'] = 0,0,0
tests['year'], tests['month'], tests['date'] = 0,0,0
train.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in train['日期']])
tests.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in tests['日期']])
train['sum'] = train['A厂'] + train['B厂']
tests['sum'] = tests['A厂'] + tests['B厂']
train['frac_a'] = train['A厂']/train['sum']
tests['frac_a'] = tests['A厂']/tests['sum']
train['frac_b'] = train['B厂']/train['sum']
tests['frac_b'] = tests['B厂']/tests['sum']
train = train.drop('日期', axis = 1)
tests = tests.drop('日期', axis = 1)
dif_col = set(train.columns.to_list()) - set(pd.DataFrame(train.describe()).columns.to_list())
train['year'], train['month'], train['date'] = [train[i].astype(int) for i in list(dif_col)]
tests['year'], tests['month'], tests['date'] = [tests[i].astype(int) for i in list(dif_col)]

In [5]:
# train.to_csv("./data/train_modified_yan.csv", index = False)
# tests.to_csv("./data/tests_modified_yan.csv", index = False)


,A厂,B厂,year,month,date,sum,frac_a,frac_b
0,200692.0,189490.0,1,1,2018,390182.0,0.514355,0.485645
1,218365.0,211147.0,1,2,2018,429512.0,0.508403,0.491597
2,224501.0,217767.0,1,3,2018,442268.0,0.507613,0.492387
3,216431.0,212636.0,1,4,2018,429067.0,0.504422,0.495578
4,200447.0,209427.0,1,5,2018,409874.0,0.489045,0.510955
...,...,...,...,...,...,...,...,...
1030,281198.0,241711.0,10,27,2020,522909.0,0.537757,0.462243
1031,275598.0,240281.0,10,28,2020,515879.0,0.534230,0.465770
1032,268544.0,237466.0,10,29,2020,506010.0,0.530709,0.469291
1033,285484.0,237611.0,10,30,2020,523095.0,0.545759,0.454241


In [5]:
# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# R_train = pd.read_csv("./data/R_train_modified_yan.csv")
# R_tests = pd.read_csv("./data/R_tests_modified_yan.csv")

In [18]:
wee = list(list(np.repeat(1, 5))+list(np.repeat(0,2)))
wee = wee*148
train['wee'] = wee[0:1035]
train3 = pd.concat([train, train], axis = 0)
train3.iloc[1035:,0] = train3.iloc[:1035,1] # move B chang
train3.iloc[:1035,1] = 0
train3.iloc[1035:,1] = 1

In [39]:
# tests # sunday start
wee = list([0,1,1,1,1,1,0])
wee = wee*22
tests['wee'] = wee[0:151]

tests3 = pd.concat([tests, tests], axis = 0)
tests3.iloc[151:,0] = tests3.iloc[:151,1] # move B chang
tests3.iloc[:151,1] = 0
tests3.iloc[151:,1] = 1

In [41]:
train3 = train3.drop(['sum', 'frac_a', 'frac_b'], axis = 1)
tests3 = tests3.drop(['sum', 'frac_a', 'frac_b'], axis = 1)

In [46]:
train3.to_csv('./data/raw_train.csv', index = False)
tests3.to_csv('./data/raw_tests.csv', index=False)

In [48]:
import optuna

---

# tensorflow

In [3]:
target_train = train.iloc[:,0:2]
target_test  = tests.iloc[:,0:2]
tensor_train = train.iloc[:,2:]
tensor_test  = tests.iloc[:,2:]

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(tensor_train, target_train, test_size=0.25, random_state=1026529)

In [5]:
# x_train
input_shape =  [7]
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import kerastuner as kt
early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=10, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [6]:
# def model_builder():
#     model = keras.Sequential()
#     layers.Dense(1186, activation='relu', input_shape=input_shape),

#     # hp_units  = hp.Int('units', min_value=593, max_value=1186, step=32)
#     # model.add(keras.layers.Dense(units=hp_units, activation='relu'))
#     # model.add(keras.layers.BatchNormalization()),
#     model.add(keras.layers.Dense(units=593, activation='relu'))
#     model.add(keras.layers.BatchNormalization()),

#     model.add(keras.layers.Dense(units=593, activation='relu'))
#     model.add(keras.layers.BatchNormalization()),
#     model.add(keras.layers.Dense(2))

#     # hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3])
#     model.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-1),
#                 loss=keras.losses.MeanAbsolutePercentageError())
# # tf.keras.losses.MeanAbsoluteError
#     return model

In [7]:
model = tf.keras.Sequential([
      layers.Dense(1186, activation='relu', input_shape=input_shape),
      keras.layers.Dense(units=593, activation='relu'),
      keras.layers.BatchNormalization(),
      keras.layers.Dense(units=593, activation='relu'),
      keras.layers.BatchNormalization(),
      keras.layers.Dense(1)
  ])
# model.add()
model.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-1),
                loss=keras.losses.MeanAbsolutePercentageError())

In [8]:
model.fit(tensor_train, epochs=120)

In [42]:
# model.predict(tensor_test)

# H2o

In [5]:
train3 = pd.concat([train.drop('B厂', axis=1), train.drop('A厂', axis=1)], axis = 0, ignore_index=True)

In [6]:
train3.iloc[:1035,7] = False
train3.iloc[1035:,0] = train3.iloc[1035:,7]
train3.iloc[1035:,7] = True

In [8]:
train3

,A厂,year,month,date,sum,frac_a,frac_b,B厂
0,200692.0,2018,1,1,390182.0,0.514355,0.485645,False
1,218365.0,2018,1,2,429512.0,0.508403,0.491597,False
2,224501.0,2018,1,3,442268.0,0.507613,0.492387,False
3,216431.0,2018,1,4,429067.0,0.504422,0.495578,False
4,200447.0,2018,1,5,409874.0,0.489045,0.510955,False
...,...,...,...,...,...,...,...,...
2065,241711.0,2020,10,27,522909.0,0.537757,0.462243,True
2066,240281.0,2020,10,28,515879.0,0.534230,0.465770,True
2067,237466.0,2020,10,29,506010.0,0.530709,0.469291,True
2068,237611.0,2020,10,30,523095.0,0.545759,0.454241,True


In [7]:
train3.to_csv('./data/full_train.csv', index = False)

In [1]:
!MYENV='siming' 

In [2]:
# !git clone https://github.com/jenv/jenv.git ~/.jenv 
!source ~/.bash_profile

In [3]:
!source activate siming

zsh:source:1: no such file or directory: activate


In [4]:
# %env
!which python

/usr/local/bin/python


In [18]:
!jupyter kernelspec list

Available kernels:
  pycharm-cc37a424                                                                         /Users/fyenne/Library/Jupyter/kernels/pycharm-cc37a424
  python374jvsc74a57bd0e774977668b7c0ae8309835a5187aa7fbf7669e7d0bb59755bc63e573643edcd    /Users/fyenne/Library/Jupyter/kernels/python374jvsc74a57bd0e774977668b7c0ae8309835a5187aa7fbf7669e7d0bb59755bc63e573643edcd
  python386jvsc74a57bd0ce909994499a62a89f8945d91d57c51a920a5cb8f3e61b9dc99d2a0ea5862498    /Users/fyenne/Library/Jupyter/kernels/python386jvsc74a57bd0ce909994499a62a89f8945d91d57c51a920a5cb8f3e61b9dc99d2a0ea5862498
  python386jvsc74a57bd0ff684bfe6bac5aee95fe25613146015247c6b1b77f36ee2373fa7468888d9e66    /Users/fyenne/Library/Jupyter/kernels/python386jvsc74a57bd0ff684bfe6bac5aee95fe25613146015247c6b1b77f36ee2373fa7468888d9e66
  python392jvsc74a57bd057c1e8c2cf22d1afad147312b0d4064398b1b8f3e9dbd8f3a3abf6cddbe75da1    /Users/fyenne/Library/Jupyter/kernels/python392jvsc74a57bd057c1e8c2cf22d1afad147312b0d4064398b1b8f3

In [5]:
%env PATH=/Users/fyenne/miniforge3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin

env: PATH=/Users/fyenne/miniforge3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin


In [2]:
!java -version

java version "14.0.2" 2020-07-14
Java(TM) SE Runtime Environment (build 14.0.2+12-46)
Java HotSpot(TM) 64-Bit Server VM (build 14.0.2+12-46, mixed mode, sharing)


In [20]:
!/usr/libexec/java_home -V

Matching Java Virtual Machines (7):
    16 (arm64) "UNDEFINED" - "OpenJDK 16" /opt/homebrew/Cellar/openjdk/15.0.2/libexec/openjdk.jdk/Contents/Home
    14.0.2 (x86_64) "Oracle Corporation" - "Java SE 14.0.2" /Library/Java/JavaVirtualMachines/jdk-14.0.2.jdk/Contents/Home
    11.0.2 (x86_64) "Oracle Corporation" - "Java SE 11.0.2" /Library/Java/JavaVirtualMachines/jdk-11.0.2.jdk/Contents/Home
    1.8.291.10 (x86_64) "Oracle Corporation" - "Java" /Library/Internet Plug-Ins/JavaAppletPlugin.plugin/Contents/Home
    1.8.0_291 (x86_64) "Oracle Corporation" - "Java SE 8" /Library/Java/JavaVirtualMachines/jdk1.8.0_291.jdk/Contents/Home
    1.8.0_251 (x86_64) "Oracle Corporation" - "Java SE 8" /Library/Java/JavaVirtualMachines/jdk1.8.0_251.jdk/Contents/Home
    1.8.0_201 (x86_64) "Oracle Corporation" - "Java SE 8" /Library/Java/JavaVirtualMachines/jdk1.8.0_201.jdk/Contents/Home
/opt/homebrew/Cellar/openjdk/15.0.2/libexec/openjdk.jdk/Contents/Home


In [26]:
# !source activate siming
!source ~/.bash_profile

In [27]:
!jenv add /Library/Java/JavaVirtualMachines/jdk-14.0.2.jdk/Contents/Home

zsh:1: command not found: jenv


In [28]:
!jenv global 14.0

zsh:1: command not found: jenv


In [29]:
!export JAVA_HOME=`/usr/libexec/java_home -v 1.8.0_291`

---

In [1]:
import h2o
h2o.init(nthreads=-1, max_mem_size = "8G")

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,30 mins 59 secs
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,3 months and 3 days
H2O_cluster_name:,H2O_from_python_fyenne_do6pnu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
# 5.4 日 # 生成 sum 和 div后, 联合额外的pred sum div 去预测 ab厂

In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

path = '/Users/fyenne/Downloads/booooks/semester5/pythonAAE/open_data_com/data'
train = pd.read_csv(path+'/train.csv')
tests = pd.read_csv(path+'/test.csv')
sub = pd.read_csv(path+'/sub.csv')

train['year'], train['month'], train['date'] = 0,0,0
tests['year'], tests['month'], tests['date'] = 0,0,0
train.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in train['日期']])
tests.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in tests['日期']])


In [6]:

train = train.drop('日期', axis = 1)
tests = tests.drop('日期', axis = 1)
dif_col = set(train.columns.to_list()) - set(pd.DataFrame(train.describe()).columns.to_list())
train['year'], train['month'], train['date'] = [train[i].astype(int) for i in list(dif_col)]
tests['year'], tests['month'], tests['date'] = [tests[i].astype(int) for i in list(dif_col)]

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A厂      1035 non-null   float64
 1   B厂      1035 non-null   float64
 2   year    1035 non-null   int64  
 3   month   1035 non-null   int64  
 4   date    1035 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 40.6 KB


In [8]:
train_raw = train # back up
tests_raw = tests

In [9]:
train_raw['sum'] = train_raw['A厂'] + train_raw['B厂']
tests_raw['sum'] = tests_raw['A厂'] + tests_raw['B厂']
train_raw['sub'] = abs(train_raw['A厂'] - train_raw['B厂'])
tests_raw['sub'] = abs(tests_raw['A厂'] - tests_raw['B厂'])
train_raw['div'] = train_raw['A厂'] / train_raw['B厂']
tests_raw['div'] = tests_raw['A厂'] / tests_raw['B厂']
import math
train_raw['log'] = pd.DataFrame([math.log(i) for i in train_raw['A厂']]) / pd.DataFrame([math.log(i) for i in train_raw['B厂']])
tests_raw['log'] = 1

In [10]:
train = train_raw # back up
tests = tests_raw

In [11]:
bs_train = train_raw[list(train_raw.columns[2:5])]
bs_tar   = train_raw[list(train_raw.columns[5: ])]

In [12]:
train

,A厂,B厂,year,month,date,sum,sub,div,log
0,200692.0,189490.0,2018,1,1,390182.0,11202.0,1.059117,1.004726
1,218365.0,211147.0,2018,1,2,429512.0,7218.0,1.034185,1.002742
2,224501.0,217767.0,2018,1,3,442268.0,6734.0,1.030923,1.002478
3,216431.0,212636.0,2018,1,4,429067.0,3795.0,1.017847,1.001442
4,200447.0,209427.0,2018,1,5,409874.0,8980.0,0.957121,0.996423
...,...,...,...,...,...,...,...,...,...
1030,281198.0,241711.0,2020,10,27,522909.0,39487.0,1.163365,1.012207
1031,275598.0,240281.0,2020,10,28,515879.0,35317.0,1.146982,1.011069
1032,268544.0,237466.0,2020,10,29,506010.0,31078.0,1.130873,1.009936
1033,285484.0,237611.0,2020,10,30,523095.0,47873.0,1.201476,1.014828


In [13]:
bs_tests = tests_raw[list(tests_raw.columns[2:5])]

In [14]:
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn import model_selection
cv_split = model_selection.ShuffleSplit(n_splits = 6, test_size = .35, train_size = .65, random_state = 1026529)
alg1 = Lasso()
cv_results_1 = model_selection.cross_validate(alg1, bs_train, bs_tar, cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_1['test_score'].mean()

-0.8122884277084048

In [22]:
from sklearn.ensemble import RandomForestRegressor
alg2 = RandomForestRegressor(n_estimators = 11, min_samples_split = 3, min_samples_leaf=5)
cv_results_2 = model_selection.cross_validate(alg2, bs_train, bs_tar, cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_2['test_score'].mean()

-0.4656554269579531

In [23]:
bs_train.info

<bound method DataFrame.info of       year  month  date
0     2018      1     1
1     2018      1     2
2     2018      1     3
3     2018      1     4
4     2018      1     5
...    ...    ...   ...
1030  2020     10    27
1031  2020     10    28
1032  2020     10    29
1033  2020     10    30
1034  2020     10    31

[1035 rows x 3 columns]>

In [25]:
from sklearn.neural_network import MLPRegressor
alg3 = MLPRegressor(hidden_layer_sizes = (345, 3), max_iter=1000, activation = 'identity', solver = 'adam', learning_rate = 'adaptive')
cv_results_3 = model_selection.cross_validate(alg3, bs_train, bs_tar, cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_3['test_score'].mean()

/Users/fyenne/miniforge3/envs/siming/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/fyenne/miniforge3/envs/siming/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/fyenne/miniforge3/envs/siming/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/fyenne/miniforge3/envs/siming/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
 

-4.300628158638928

In [84]:
fit1 = alg2.fit(bs_train,bs_tar)
fit1_pre = pd.DataFrame(fit1.predict(bs_tests))

In [85]:
fit1_pre

,0,1,2,3
0,327937.79,9505.29,0.943754,0.995152
1,325938.41,13655.43,0.919484,0.992974
2,328990.20,12137.84,0.929069,0.993862
3,339625.40,10400.52,0.941192,0.994972
4,354218.91,9503.81,0.950571,0.995781
...,...,...,...,...
146,436631.37,102284.07,1.614388,1.039749
147,425570.07,103591.03,1.644571,1.041385
148,426341.80,108809.12,1.686190,1.043498
149,418934.40,100801.86,1.636130,1.041005


---

## what u need in final move has done above.

In [243]:
train_raw = train_raw.drop(['A厂','B厂'], axis = 1)
tests_raw = tests_raw.drop(['A厂','B厂'], axis = 1)

In [244]:
list(train_raw.columns)[:-1]

['year', 'month', 'date']

In [245]:
train_list=list(train_raw.columns)[:-1]
train_1=h2o.H2OFrame(train_raw)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [246]:
train_1

year,month,date,log
2018,1,1,1.00473
2018,1,2,1.00274
2018,1,3,1.00248
2018,1,4,1.00144
2018,1,5,0.996423
2018,1,6,1.00498
2018,1,7,1.00067
2018,1,8,1.00347
2018,1,9,1.00272
2018,1,10,0.998692


# train!

In [73]:
param = {
      "ntrees" : 1060
    , "max_depth" : 15
    , "learn_rate" : 0.05
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4240
    , "score_tree_interval": 100
    , "stopping_metric" :"MSE"
    , "nfolds":8
    , "fold_assignment":"AUTO"
    , "keep_cross_validation_predictions" : True
    # , "booster":"dart"
}

from h2o.estimators import H2OXGBoostEstimator
model_xgb = H2OXGBoostEstimator(**param)
model_xgb.train(x = train_list, y = 'log', training_frame = train_1)
# param={
#     "ntrees" : 1060
#     , "max_depth" : 15
#     , "learn_rate" : 0.02
#     , "sample_rate" : 0.7
#     , "col_sample_rate_per_tree" : 0.9
#     , "min_rows" : 5
#     , "seed": 4240
#     , "score_tree_interval": 100
#     , "stopping_metric" :"RMSE"
#     , "nfolds":8
#     , "fold_assignment":"AUTO"
#     , "keep_cross_validation_predictions" : True
#  }

In [248]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator  # import gbm estimator
model_gbm = H2OGradientBoostingEstimator(**param)
model_gbm.train(x = train_list, y = 'log', training_frame = train_1)

/Users/fyenne/miniforge3/envs/siming/lib/python3.8/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Stopping metric is ignored for _stopping_rounds=0.
  warnings.warn(mesg["message"], RuntimeWarning)
gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [202]:
from h2o.estimators import H2OStackedEnsembleEstimator
stack = H2OStackedEnsembleEstimator(model_id="ensemble11",
                                       training_frame=train_1,
                                       #validation_frame=test,
                                       base_models=[model_xgb.model_id,model_gbm.model_id],
                                    metalearner_algorithm="glm")
stack.train(x=train_list, y="log", training_frame=train_1)

stackedensemble Model Build progress: |███████████████████████████████████| 100%


In [204]:
tests_raw

,year,month,date,sub
0,2020,11,01,-37
1,2020,11,02,187
2,2020,11,03,-615
3,2020,11,04,-453
4,2020,11,05,-254
...,...,...,...,...
146,2021,03,27,-739
147,2021,03,28,-132
148,2021,03,29,-278
149,2021,03,30,201


In [249]:
test_1=h2o.H2OFrame(tests_raw) #### 
pred1=model_xgb.predict(test_1)
pred2=model_gbm.predict(test_1)
pred_df1=pred1.as_data_frame()
pred_df2=pred2.as_data_frame()

weights = {
    'lgb_model': 9,
    'xgb_model': 1
}

final_preds = (pred2 * weights['lgb_model'] + pred1 * weights['xgb_model']) / (weights['lgb_model'] + weights['xgb_model'])

Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


In [250]:
final_preds

predict
0.99243
0.996435
1.00053
0.997961
0.998204
1.00501
1.00177
1.01281
1.0138
1.01623


In [207]:
test_1=h2o.H2OFrame(tests_raw)
pred3 = stack.predict(test_1)

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


# three vars:

In [81]:
# sum_pred = h2o.as_list(pred3)
sum_in_tests = pd.concat([sum_pred, sum_pred], axis = 0)

In [213]:
sub_pred = h2o.as_list(final_preds)
sub_in_tests = pd.concat([sub_pred, sub_pred], axis = 0)

In [82]:
# div_pred = h2o.as_list(pred3)
div_in_tests = pd.concat([div_pred, div_pred], axis = 0)

In [251]:
log_pred = h2o.as_list(final_preds)
log_in_tests = pd.concat([log_pred, log_pred], axis = 0)

In [151]:
h2o_extra_predict = h2o.as_list(pred3)

In [152]:
h2o_extra_predict = h2o_extra_predict['predict']

In [153]:
h2o_extra_predict

0      215864.008289
1      215942.855956
2      216418.497391
3      216292.699603
4      216109.361190
           ...      
297    213976.461382
298    213924.097302
299    213895.186416
300    214272.057598
301    213906.029365
Name: predict, Length: 302, dtype: float64

---

# feed sum back to train

In [253]:
train_raw2 = pd.read_csv('/Users/fyenne/Downloads/booooks/semester5/pythonAAE/open_data_com/data/raw_train.csv')
tests_raw2 = pd.read_csv('/Users/fyenne/Downloads/booooks/semester5/pythonAAE/open_data_com/data/raw_tests.csv')

In [270]:
train_raw2
# tests_raw2

,A厂,B厂,year,month,date,wee,sum,div,sub,log
0,200692.0,0.0,1,1,2018,1,390182.0,1.059117,11202.0,1.004726
1,218365.0,0.0,1,2,2018,1,429512.0,1.034185,7218.0,1.002742
2,224501.0,0.0,1,3,2018,1,442268.0,1.030923,6734.0,1.002478
3,216431.0,0.0,1,4,2018,1,429067.0,1.017847,3795.0,1.001442
4,200447.0,0.0,1,5,2018,1,409874.0,0.957121,-8980.0,0.996423
...,...,...,...,...,...,...,...,...,...,...
2065,241711.0,1.0,10,27,2020,1,522909.0,1.163365,39487.0,1.012207
2066,240281.0,1.0,10,28,2020,1,515879.0,1.146982,35317.0,1.011069
2067,237466.0,1.0,10,29,2020,1,506010.0,1.130873,31078.0,1.009936
2068,237611.0,1.0,10,30,2020,1,523095.0,1.201476,47873.0,1.014828


In [263]:
train_raw2['sum'] = list(pd.concat([train_raw['sum'],train_raw['sum']], axis = 0))
train_raw2['div'] = list(pd.concat([train_raw['div'],train_raw['div']], axis = 0))
train_raw2['sub'] = list(pd.concat([train_raw['sub'],train_raw['sub']], axis = 0))
train_raw2['log'] = list(pd.concat([train_raw['log'],train_raw['log']], axis = 0))
tests_raw2['sum'] = list(sum_in_tests['predict'])
tests_raw2['div'] = list(div_in_tests['predict'])
tests_raw2['sub'] = list(sub_in_tests['predict'])
tests_raw2['log'] = list(log_in_tests['predict'])

In [272]:
train_list=list(train_raw2.columns)[1:]
train_1=h2o.H2OFrame(train_raw2)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [274]:
train_1

A厂,B厂,year,month,date,wee,sum,div,sub,log
200692,0,1,1,2018,1,390182,1.05912,11202,1.00473
218365,0,1,2,2018,1,429512,1.03418,7218,1.00274
224501,0,1,3,2018,1,442268,1.03092,6734,1.00248
216431,0,1,4,2018,1,429067,1.01785,3795,1.00144
200447,0,1,5,2018,1,409874,0.957121,-8980,0.996423
197836,0,1,6,2018,0,384063,1.06234,11609,1.00498
198552,0,1,7,2018,0,395477,1.00826,1627,1.00067
197743,0,1,8,2018,1,387311,1.04312,8175,1.00347
191391,0,1,9,2018,1,376574,1.03352,6208,1.00272
197122,0,1,10,2018,1,397418,0.984153,-3174,0.998692


In [275]:
param = {
      "ntrees" : 2070
    , "max_depth" : 15
    , "learn_rate" : 0.01
    # , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4240
    , "score_tree_interval": 100
    , "stopping_metric" :"MSE"
    , "nfolds":8
    , "fold_assignment":"AUTO"
    , "keep_cross_validation_predictions" : True
    # , "booster":"dart"
}

from h2o.estimators import H2OXGBoostEstimator
model_xgb = H2OXGBoostEstimator(**param)
model_xgb.train(x = train_list, y = 'A厂', training_frame = train_1)

/Users/fyenne/miniforge3/envs/siming/lib/python3.8/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Stopping metric is ignored for _stopping_rounds=0.
  warnings.warn(mesg["message"], RuntimeWarning)
xgboost Model Build progress: |███████████████████████████████████████████| 100%


In [276]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator  # import gbm estimator
model_gbm = H2OGradientBoostingEstimator(**param)
model_gbm.train(x = train_list, y = 'A厂', training_frame = train_1)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [278]:
test_1=h2o.H2OFrame(tests_raw2) #### 
pred1=model_xgb.predict(test_1)
pred2=model_gbm.predict(test_1)
pred_df1=pred1.as_data_frame()
pred_df2=pred2.as_data_frame()

weights = {
    'lgb_model': 5,
    'xgb_model': 5
}

final_preds = (pred2 * weights['lgb_model'] + pred1 * weights['xgb_model']) / (weights['lgb_model'] + weights['xgb_model'])

Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


In [281]:
h2o.as_list(final_preds)

,predict
0,208607.430235
1,212087.663406
2,217484.933794
3,213004.832236
4,213907.125810
...,...
297,170200.241547
298,169949.498801
299,173865.959451
300,175512.196128


In [282]:
four_var_sub = sub
four_var_sub['A厂'] = h2o.as_list(final_preds)['predict'][:151]
four_var_sub['B厂'] = np.array(h2o.as_list(final_preds)['predict'][151:])
four_var_sub

,日期,A厂,B厂
0,2020/11/01,208607.430235,218149.521470
1,2020/11/02,212087.663406,214417.444352
2,2020/11/03,217484.933794,210193.388155
3,2020/11/04,213004.832236,211813.280467
4,2020/11/05,213907.125810,213306.723670
...,...,...,...
146,2021/03/27,261283.009972,170200.241547
147,2021/03/28,260716.930657,169949.498801
148,2021/03/29,258132.864299,173865.959451
149,2021/03/30,257303.799480,175512.196128


In [283]:
four_var_sub.to_csv('./data/4_var.csv', index=False)

In [144]:
bssub = sub

In [154]:
bssub['A厂'] = h2o_extra_predict[:151]
bssub['B厂'] = h2o_extra_predict[151:]

In [156]:
bssub['B厂'] = np.array(h2o_extra_predict[151:]);bssub['B厂']

0      215827.279169
1      215763.857356
2      215507.569231
3      215575.478670
4      215691.834521
           ...      
146    213976.461382
147    213924.097302
148    213895.186416
149    214272.057598
150    213906.029365
Name: B厂, Length: 151, dtype: float64

In [158]:
bssub.to_csv('./data/h2o_extra_sub.csv', index=False)

In [159]:
bssub

,日期,A厂,B厂
0,2020/11/01,215864.008289,215827.279169
1,2020/11/02,215942.855956,215763.857356
2,2020/11/03,216418.497391,215507.569231
3,2020/11/04,216292.699603,215575.478670
4,2020/11/05,216109.361190,215691.834521
...,...,...,...
146,2021/03/27,218158.815020,213976.461382
147,2021/03/28,218281.091822,213924.097302
148,2021/03/29,218198.176717,213895.186416
149,2021/03/30,218055.381674,214272.057598


In [175]:
bsub2 = bssub
bsub_gbm = bssub
bsub_mixin = bssub

In [176]:
bsub2['A厂'] = pred_df1[:151]
bsub2['B厂'] = np.array(pred_df1[151:])
bsub2

,日期,A厂,B厂
0,2020/11/01,215426.828125,214070.031250
1,2020/11/02,217470.593750,213393.656250
2,2020/11/03,223888.859375,207705.046875
3,2020/11/04,222198.906250,208902.546875
4,2020/11/05,220658.812500,210942.937500
...,...,...,...
146,2021/03/27,259832.296875,174713.703125
147,2021/03/28,262186.281250,173670.296875
148,2021/03/29,261317.875000,173198.921875
149,2021/03/30,259113.109375,181506.640625


In [177]:
bsub_gbm['A厂'] = pred_df2[:151]
bsub_gbm['B厂'] = np.array(pred_df2[151:])
bsub_gbm

,日期,A厂,B厂
0,2020/11/01,210811.904971,210659.674863
1,2020/11/02,211997.097326,208750.525777
2,2020/11/03,224985.654668,203955.220371
3,2020/11/04,221542.976922,205531.923032
4,2020/11/05,215613.920609,208244.705812
...,...,...,...
146,2021/03/27,260200.445888,174325.943912
147,2021/03/28,262844.047829,173228.745272
148,2021/03/29,260332.436805,172519.532510
149,2021/03/30,256707.603529,179627.695066


In [186]:
bsub_mixin['A厂'] = h2o.as_list(final_preds)['predict'][:151]
bsub_mixin['B厂'] = np.array(h2o.as_list(final_preds)['predict'][151:])

In [187]:
bsub_mixin.to_csv('./data/mixin_pre2.csv', index=False)